## Daily Female Births

In [1]:
# importing libraries

import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# cheching the dataset

dataset = pd.read_csv('daily-total-female-births.csv')
dataset.sample(8)


,Date,Births
314,1959-11-11,40
159,1959-06-09,46
160,1959-06-10,42
111,1959-04-22,45
228,1959-08-17,33
360,1959-12-27,37
86,1959-03-28,36
307,1959-11-04,42


In [3]:
series = pd.read_csv('daily-total-female-births.csv', header = 0, index_col = 0, parse_dates = True, squeeze = True)
series

Date
1959-01-01    35
1959-01-02    32
1959-01-03    30
1959-01-04    31
1959-01-05    44
              ..
1959-12-27    37
1959-12-28    52
1959-12-29    48
1959-12-30    55
1959-12-31    50
Name: Births, Length: 365, dtype: int64

In [4]:
# number of observation
print(series.size)

365
